<img align="center" width="12%" style="padding-right:10px;" src="Images/Ccom.png">

In [15]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np

sys.path.append(os.getcwd())  # add the current folder to the list of paths where Python looks for modules 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Lab 4.1: Preparing Positioning Data for Hot-Linking to Video Data<a href="https://piazza.com/class/jzvaaav18cf2j7"><img src="Images/help.png"  title="Ask questions on Piazza.com" align="right" width="10%" alt="Piazza.com\"></a><br><br> 

In this Notebook you will update the Position class from the Integrated Seabed Mapping Systems course Lab A 
assignment. The updates consists of adding a special write method. For the purpose of creating hot links between positions and video frames we need a synchronization method that allows for the linking of positions with the appropriate video frame.

The video data does not contain positions thus we need to get the positioning data from an external source. In this particular case the source is HYPACK RAW files, for which we wrote a parser in step 4.0. The problem with this is that there is no good synchronization mechanism between the positioning data and the video data. All we know is that the video data and positioning data are supposed to start and end simultaneously as the operator was instructed to start and stop logging for both the video and HYPACK. However, reality is that the length of the videos and associated HYPACK data files do not match. As there were no notes in the data acquisition log addressing this issue we have no information on how to address the issue. 

<img align="center" width="80%" style="padding-right:10px;" src="Images/integration.png">

The best that we can do is to average the error over the duration of the data - that is to *stretch* the data in such a way that the durations of the data files match. The mechanism that we will use for this is one of linear scaling.

<img align="center" width="80%" style="padding-right:10px;" src="Images/linearscaling.png">

The manner in which the hotlink in ArcMAP will be implemented is that you will visualize all the positions in the HYPACK data files. To play the associated video you will activate the hotlink by clicking on the location. Thus, to get the correct frame we need to know what epoch is associated to the position, but we already know that we cannot simply take the time from the start (or could we?). What we can say is that at the start we are at fraction 0/1 of the length of both data streams, and at the end we are at fraction 1/1, thus logically at the halfway point we are at $(NFrames/2)/NFrames = 1/2$ of the video frames. An easy way to then achieve the time scaling is to determine the duration of a HYPACK file, determine for each position its fraction of the duration of the file and, show the frame located at the fraction of the video.
<br><br>
$$duration=t_{pos}(end)-t_{pos}(start)$$
<br><br>
$$fraction=\frac{t_{pos}(end)-t_{pos}(current)}{duration}$$
<br><br>

## 4.1.0  Creating a Data File for Hotlinking Positions to Video
I have created some software that will play one second of a video starting at the fraction provided as input argument. This means that to implement the synchronization we do not even have to read the video data file, which will make this step a lot easier for you. All you need to do is to parse the HYPACK file and write the positions with the associated fraction to an output file that ArcMAP can read.

## 4.1.1 Create a Write Method

Add a method named `write_hotlink()` to your `Position` class. This method should take the arguments `self`, and `hotlink_path`

## 4.1.2 Create an Output File name

In the `write_hotlink()` add a variable fullpath and assign it the value of the name of the position input file i.e., `self.full_path` but without the file extension. You may use the `os.path.splitext` method for this purpose.

Now add the string "\_pos.txt" to the end of the string contained in `fullpath`

## 4.1.3 Verify the Paths

Use `os.path.exists(fullpath)` to see whether an output file by the name contained in `fullpath`. If it does execute the command `print('Overwriting file: ' + fullpath)`, else execute `print('Writing to file: ' + fullpath)`

Also check whether the argument `hotlink_path` is a string. Throw an exception if it is not

## 4.1.4 Open  an Ouput file for Writing

open an output file of the name contained in the string `fullpath` in writing mode.

## 4.1.5 Write a Header Line

This data file will have five columns of data The first being the date, then the time, then the latitude, than the longitude and finally a string comprised of the pathname and fraction separated by a backslash. Note that the other data associated to the positionis are not all that relevant. The height is immaterial as we use the positions for an underwater video that is only loosely coupled to the positioning. The quality is of no concern for the same reason. Including the height and quality would actually be misleading, giving more credence to the positioning data than it deserves.

## 4.1.6 Writing the Records

    date time latitude longitude path
    2012-07-03 15:34:56 043.05206333 -070.64148167 Video\v_3.abc?0.000000
    2012-07-03 15:34:57 043.05206167 -070.64148000 Video\v_3.abc?0.013333
    2012-07-03 15:34:58 043.05206167 -070.64147500 Video\v_3.abc?0.026667


For each of the positions in the `Position` object write the date and time of collection, the longitude (12 characters with leading zeros) and latitude (13 characters with leading zeros) in signed decimal degrees with 8 decimals. and finally a combination of the relative path to the video file to be played from the ArcMAP project folder to the video folder combined with the fraction of the video file that we want to see when clicking on the position in ArcMAP. Above you see an example of the first 4 lines of the file that results from processing the data contained in the 999_1029.RAW HYPACK RAW file. The `write_hotlink()` method was called with the argument `hotlink_path = " "Video\\v_3.abc"`. Note the use of backslashes, which is how paths are defined on Windows machines (as opposed to the Linux base ePOM server which uses forward slashes). Also, note that there is two backslashes separating the folder from the file - this is because the `\` character is an operator in Python - to indicate that we want the character rather than the operator we use `\\`.

In [41]:
from mycode.position import Position

# get the absolute path to the current directory

abs_path=os.path.abspath(os.path.curdir)

# Instantiate a Position object and read data into it 

positions=Position()
positions.read_hypack_raw_file(abs_path+'/Data/000_1133.RAW')

# Provide the relative path  - note that this needs to be aWindows style path i.e., with backslashes
positions.write_hotlink(hotlink_path = "Video\\v_3.abc")

Opening GNSS data file:/home/jupyter-semmed/ESCI_872/Data/000_1133.RAW
HYPACK RAW Header start time and date: Tue Jul  3 11:33:37 2012
HYPACK RAW Header start time and date in UTC: Tue Jul  3 15:33:37 2012
HYPACK RAW file contains: 76 GGA records
Overwriting file: /home/jupyter-semmed/ESCI_872/Data/000_1133_pos.txt


## 4.1.7 Downloading the data file

If everything went well You have now created the data file *000_1133_pos.txt* which is stored in your Data directory on the ePOM server. You may download this file 


In [ ]:
positions.draw()

<img align="left" width="5%" style="padding-right:10px;" src="Images/email.png">

*For issues or suggestions related to this notebook that should not be addressed on Piazza, write to: semmed@ccom.unh.edu*